# Transformers Interpret Multiclass Classification Example

In [12]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## Import Industry Classification Mode
This finetuned model by @sampathkethineedi uses a distilbert base to predict the professional industry a text is referring to. 

In [13]:
tokenizer = AutoTokenizer.from_pretrained("sampathkethineedi/industry-classification")
model = AutoModelForSequenceClassification.from_pretrained(
    "sampathkethineedi/industry-classification"
)

Let's explore the classes, there are 62 unique classes. Many of these are overlapping/related industries such as __Health Care Equipment__ and __Health Care Supplies__.

In [14]:
model.config.id2label

{0: 'Advertising',
 1: 'Aerospace & Defense',
 2: 'Apparel Retail',
 3: 'Apparel, Accessories & Luxury Goods',
 4: 'Application Software',
 5: 'Asset Management & Custody Banks',
 6: 'Auto Parts & Equipment',
 7: 'Biotechnology',
 8: 'Building Products',
 9: 'Casinos & Gaming',
 10: 'Commodity Chemicals',
 11: 'Communications Equipment',
 12: 'Construction & Engineering',
 13: 'Construction Machinery & Heavy Trucks',
 14: 'Consumer Finance',
 15: 'Data Processing & Outsourced Services',
 16: 'Diversified Metals & Mining',
 17: 'Diversified Support Services',
 18: 'Electric Utilities',
 19: 'Electrical Components & Equipment',
 20: 'Electronic Equipment & Instruments',
 21: 'Environmental & Facilities Services',
 22: 'Gold',
 23: 'Health Care Equipment',
 24: 'Health Care Facilities',
 25: 'Health Care Services',
 26: 'Health Care Supplies',
 27: 'Health Care Technology',
 28: 'Homebuilding',
 29: 'Hotels, Resorts & Cruise Lines',
 30: 'Human Resource & Employment Services',
 31: 'IT Co

Import __SequenceClassificationExplainer__ from transformers interpret. This class should work with most if not all language models with a sequence classification head from the transformers package. 


In [15]:
from transformers_interpret import SequenceClassificationExplainer

In [16]:
sample_text = """
Stocks ended a choppy session mixed as investors digested a host of corporate earnings results and considered policymakers’ next moves to support the still virus-stricken economy.
The S&P 500 shook off earlier declines to narrowly eke out a record closing high.The Dow ended a tick below its recent record closing level."""

In [17]:
multiclass_explainer = SequenceClassificationExplainer(model=model, tokenizer=tokenizer)

In [18]:
# call the exlplainer
word_attributions = multiclass_explainer(text=sample_text)

In [19]:
# seems to be an appropriate prediction
multiclass_explainer.predicted_class_name

'Investment Banking & Brokerage'

In [20]:
# look the the raw word attributions
word_attributions

[('[CLS]', 0.0),
 ('stocks', 0.17552720036501643),
 ('ended', -0.03850328798468155),
 ('a', 0.005213925021134272),
 ('chop', -0.37112166870328483),
 ('##py', 0.054741270139833334),
 ('session', 0.11222943583103878),
 ('mixed', 0.045234820681212935),
 ('as', 0.008543612868500418),
 ('investors', 0.22552280260575677),
 ('digest', 0.09217359100492979),
 ('##ed', 0.14644402808281493),
 ('a', -0.10249595339350155),
 ('host', -0.07218363176949291),
 ('of', 0.22421014487892135),
 ('corporate', 0.27895956349460976),
 ('earnings', 0.6027403740481148),
 ('results', -0.09964956678786037),
 ('and', -0.017122058095847055),
 ('considered', 0.2477727991236421),
 ('policy', 0.1037566155952272),
 ('##makers', 0.011601804007161865),
 ('’', 0.01130267463579217),
 ('next', -0.04636321001225286),
 ('moves', -0.004163994730185889),
 ('to', 0.0899134351814172),
 ('support', 0.07168823747145485),
 ('the', 0.027829641750952965),
 ('still', 0.07106036523177117),
 ('virus', 0.1065564568863362),
 ('-', -0.0730028

## Visualizating Explanations 
With a single call to the `visualize()` method we get a nice inline display of what inputs are causing the activations to fire that led to this prediction. **Note the alogirthm used to calcualte attributions are Layer Integreated Gradients to read more about them click [here](https://captum.ai/docs/algorithms)**

In [21]:
html = multiclass_explainer.visualize()

## Explaining The Same Text For A Different Class 
Lets say we think this text could also fall somewhat under the class of __Asset Management & Custody Banks__ If we want it is also possible to get an explantion/attributions for the text with that class

In [22]:
word_attributions = multiclass_explainer(sample_text, class_name="Asset Management & Custody Banks")

In [23]:
# look the the raw word attributions
word_attributions

[('[CLS]', 0.0),
 ('stocks', -0.14385808013018822),
 ('ended', 0.17111802090540412),
 ('a', -0.06059880938330258),
 ('chop', 0.5140063786258902),
 ('##py', -0.08750550380762823),
 ('session', -0.07754090851288636),
 ('mixed', 0.03618530966407318),
 ('as', -0.009426322086584097),
 ('investors', 0.4988439501492522),
 ('digest', -0.2130210252729232),
 ('##ed', -0.0642909378839095),
 ('a', 0.17772630290946734),
 ('host', 0.1580884574065216),
 ('of', -0.06705090644971788),
 ('corporate', -0.15002094874754812),
 ('earnings', -0.23341494931659376),
 ('results', 0.12625839825444482),
 ('and', 0.05719575832814555),
 ('considered', -0.11990920134276094),
 ('policy', 0.14461720492702912),
 ('##makers', 0.15514270374239364),
 ('’', -0.027600562834770716),
 ('next', 0.02122585963672142),
 ('moves', 0.10101863201395013),
 ('to', -0.05263520963770683),
 ('support', -0.0014981195440918274),
 ('the', 0.08523798846114553),
 ('still', 0.0035912303371714942),
 ('virus', 0.00824973027002683),
 ('-', -0.055

The results are close to the first visualization, a good sign that the model is generalizing well for both of these related classes

In [24]:
html = multiclass_explainer.visualize()

What if we get attributions for class that makes no sense in this context such as __Restaurants__?


In [25]:
word_attributions = multiclass_explainer(sample_text, class_name="Restaurants")

There isn't much to this prediction, it is worth nothing however that the words "choppy" had a more positive impact in this instance which seems plausible given the industry. 

In [26]:
html = multiclass_explainer.visualize()

In [27]:
word_attributions

[('[CLS]', 0.0),
 ('stocks', -0.6854859058840344),
 ('ended', -0.02903205216337953),
 ('a', 0.11090246049261213),
 ('chop', 0.006652637417738755),
 ('##py', -0.20624943153114753),
 ('session', -0.07065561556653746),
 ('mixed', -0.13288257535449538),
 ('as', 0.07492890890956264),
 ('investors', -0.12119187012995146),
 ('digest', -0.12445948275050642),
 ('##ed', -0.09281772083050198),
 ('a', 0.04926221669843679),
 ('host', 0.002458038605145266),
 ('of', -0.04180982353010906),
 ('corporate', 0.09596853145329277),
 ('earnings', 0.19780804001186278),
 ('results', 0.2649820413816817),
 ('and', 0.10765607812143425),
 ('considered', 0.08802590573220752),
 ('policy', 0.11527451534122934),
 ('##makers', -0.0979763265607011),
 ('’', -0.04952380439753834),
 ('next', -0.08798391545875107),
 ('moves', 0.016330984853300038),
 ('to', -0.013984838237578378),
 ('support', -0.0746662851801564),
 ('the', 0.018439130760925827),
 ('still', -0.07228677398653408),
 ('virus', 0.07444827768777922),
 ('-', -0.06